# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, DatasetDict

In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1+cu117
CUDA version: 11.7
cuDNN version: 8500
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.32.1
Datasets version: 2.14.4


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Thu Feb  1 07:38:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   32C    P2              57W / 250W |  10915MiB / 11264MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU  | MEM |
-------------------
|  0 |   1% | 97% |
|  1 |   0% |  0% |
|  2 |   0% |  0% |
|  3 |   0% | 93% |
|  4 | 100% |  0% |
|  5 | 100% |  0% |
|  6 |   0% |  0% |
|  7 |   0% |  0% |
GPU Usage after emptying the cache
| ID | GPU  | MEM |
-------------------
|  0 |   1% | 97% |
|  1 |   0% |  0% |
|  2 |   0% |  0% |
|  3 |   0% | 93% |
|  4 | 100% |  0% |
|  5 | 100% |  0% |
|  6 |   0% |  0% |
|  7 |   1% |  1% |


In [8]:
# Smaller and faster than bert.
model_ckpt = "distilbert-base-uncased"

epochs = 3 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [9]:
dataset_load = load_dataset("kristmh/high_vs_random")
dataset_load

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 42053
    })
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 336420
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__'],
        num_rows: 42052
    })
})

In [10]:
def remove_shortest_percent(subset, percent=0.1):
    # Calculate the length of each entry
    with_length = subset.map(lambda x: {"length": len(x["text_clean"])})
    
    # Calculate the nth percentile length
    lengths = with_length["length"]
    threshold = sorted(lengths)[int(percent * len(lengths))]
    
    # Filter out entries shorter than the threshold
    filtered_subset = with_length.filter(lambda x: x["length"] > threshold)
    return filtered_subset

# Define the percentage of entries to remove
percent = 0.2
# Apply the filtering to each subset
dataset = DatasetDict({
    'train': remove_shortest_percent(dataset_load['train'], percent),
    'test': remove_shortest_percent(dataset_load['test'], percent),
    'validate': remove_shortest_percent(dataset_load['validate'], percent)
})
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'length'],
        num_rows: 268585
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'length'],
        num_rows: 33636
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'length'],
        num_rows: 33627
    })
})

In [11]:
test_dataset = load_dataset(("kristmh/test_high_vs_med_low_priority"))
test_dataset

DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class'],
        num_rows: 22097
    })
})

In [12]:
test_dataset = DatasetDict({
    'test_separate': remove_shortest_percent(test_dataset['test_separate'], percent),

})
test_dataset

DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class', 'length'],
        num_rows: 17667
    })
})

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

## Tokenization

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [17]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [18]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/33636 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'length', 'input_ids', 'attention_mask'],
        num_rows: 268585
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'length', 'input_ids', 'attention_mask'],
        num_rows: 33636
    })
    validate: Dataset({
        features: ['Unnamed: 0', 'text_clean', 'labels', '__index_level_0__', 'length', 'input_ids', 'attention_mask'],
        num_rows: 33627
    })
})

In [19]:
test_tokenized_dataset = test_dataset.map(tokenize, batched=True)
test_tokenized_dataset

DatasetDict({
    test_separate: Dataset({
        features: ['text_clean', 'labels', 'class', 'length', 'input_ids', 'attention_mask'],
        num_rows: 17667
    })
})

In [20]:
separate_test_set = test_tokenized_dataset["test_separate"]
separate_test_set = separate_test_set.remove_columns(["text_clean", "class"])
separate_test_set

Dataset({
    features: ['labels', 'length', 'input_ids', 'attention_mask'],
    num_rows: 17667
})

In [21]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean", "__index_level_0__", "Unnamed: 0"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

Dataset({
    features: ['labels', 'length', 'input_ids', 'attention_mask'],
    num_rows: 268585
})
Dataset({
    features: ['labels', 'length', 'input_ids', 'attention_mask'],
    num_rows: 33627
})


Dataset({
    features: ['labels', 'length', 'input_ids', 'attention_mask'],
    num_rows: 33636
})

## Training a classifier

In [22]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

In [24]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.567000,0.572004,0.703036,[0.70679429 0.69918062],[0.70950248 0.69645322],[0.7041067 0.70192947]
2,0.521700,0.553452,0.715021,[0.71505456 0.71498677],[0.72709681 0.70333528],[0.7034047 0.72703079]
3,0.434200,0.582983,0.720552,[0.72714074 0.71363706],[0.7218795 0.7191377],[0.73247923 0.70821992]


TrainOutput(global_step=50361, training_loss=0.5143263214587626, metrics={'train_runtime': 15649.3218, 'train_samples_per_second': 51.488, 'train_steps_per_second': 3.218, 'total_flos': 1.0673626880406528e+17, 'train_loss': 0.5143263214587626, 'epoch': 3.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [25]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [26]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 3.0

eval_accuracy = 0.7205519374312308

eval_f1 = [0.72714074 0.71363706]

eval_loss = 0.5829834938049316

eval_precision = [0.7218795 0.7191377]

eval_recall = [0.73247923 0.70821992]

eval_runtime = 211.358

eval_samples_per_second = 159.1

eval_steps_per_second = 4.973



In [27]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [28]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 3.0

eval_accuracy = 0.720180758710905

eval_f1 = [0.72622026 0.71386879]

eval_loss = 0.5867281556129456

eval_precision = [0.72285598 0.7173581 ]

eval_recall = [0.72961599 0.71041326]

eval_runtime = 211.555

eval_samples_per_second = 158.994

eval_steps_per_second = 4.973



In [29]:
trainer.save_model(model_dir + "_local") 

In [30]:
# Evaluate separate test set
separate_test_results = trainer.evaluate(eval_dataset=separate_test_set)

In [31]:
for key, value in sorted(separate_test_results.items()):
    print(f"{key} = {value}\n")

epoch = 3.0

eval_accuracy = 0.5921775060847908

eval_f1 = [0.6608933  0.48853553]

eval_loss = 1.0119459629058838

eval_precision = [0.55908584 0.67351732]

eval_recall = [0.80803315 0.38327022]

eval_runtime = 110.8983

eval_samples_per_second = 159.308

eval_steps_per_second = 4.987



In [32]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [33]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [34]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_0', 'score': 0.6833946704864502}]

In [35]:
classifier("this is super important")

[{'label': 'LABEL_1', 'score': 0.8435372710227966}]

In [36]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.8345832824707031}]

## Important to delete large objects to free memory 
del train_dataset

In [37]:
del validation_dataset 

In [38]:
del model

In [39]:
# Free cache
torch.cuda.empty_cache()

In [40]:
!nvidia-smi

Thu Feb  1 12:09:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   31C    P2              57W / 250W |  10915MiB / 11264MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--